<a href="https://colab.research.google.com/github/andreaPorrazzo/30-Days-Of-JavaScript/blob/master/ml_03_04_classif_w_prepr_nocode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing: transform categorical data
by Claudio Sartori

In `scikit-learn` the classifiers require numeric data. The library makes available a set of preprocessing fuctions which help the transformation. This exercise proposes two types of transformations:

- `OneHotEncoder` for purely categorical columns: if the column has **V** distinct values it is substituted by **V** binary columns where in each row only the bit corrosponding to the original value is true
- `OrdinalEncoder` for ordinal columns: the original **V** values are mapped into the **0..V-1** range

The additional function `ColumnTransformer` allows to apply the different transformations to the appropriate columns with a single statement.

### To do:
- import the appropriate names
- set the random state
- import the data set with the appropriate column names
- inspect the content and the data types
- read carefully the `.names` file of the data set, to understand which are the ordinal and categorical data
- data cleaning
    - the **ordinal transformer** generates a mapping from strings to numbers according to the lexicographic sorting of the strings; in this particular case, the strings indicate numeric subranges, and ranges with one digit constitute exceptions
        '5-9' happens to be after '20-25'
    - it is necessary to transform '5-9' into '05-09', and the same for other similar cases
    - a way to do this is to prepare dictionaries for the translation and use the `.map` function
- prepare the lists of the ordinal, categorical and numeric columns
- prepare the preprocessor
- split the cleaned data into the X and y part
- fit_transform the preprocessor and generate the transformed data set
- split the transformed data set into train and test
- use the same method used for the exercise of 19/11 to test several classifiers

In [32]:
"""
http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
@author: scikit-learn.org and Claudio Sartori
"""
import warnings
warnings.filterwarnings('ignore') # uncomment this line to suppress warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

print(__doc__) # print information included in the triple quotes at the beginning

random_state = 42


http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
@author: scikit-learn.org and Claudio Sartori



In [6]:
# url = 'diagnosis.data'
# names = ['Temp', 'Nau', 'Lum', 'Uri', 'Mic', 'Bur', 'd1', 'd2']
# sep = "\t"
url = 'breast-cancer.data'
names = ['Class','age','menopause','tumor-size','inv-nodes',
         'node-caps','deg-malig','breast','breast-quad','irradiat']
sep = ","

df = pd.read_csv(url, names = names, sep=sep)
df.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


Show the types of the columns

In [7]:
print(df.dtypes)

Class          object
age            object
menopause      object
tumor-size     object
inv-nodes      object
node-caps      object
deg-malig       int64
breast         object
breast-quad    object
irradiat       object
dtype: object


Clean the column `tumor-size`

In [8]:
tumor_size_dict = dict(zip(list(df['tumor-size'].unique()),list(df['tumor-size'].unique())))
tumor_size_dict

{'0-4': '0-4',
 '10-14': '10-14',
 '15-19': '15-19',
 '20-24': '20-24',
 '25-29': '25-29',
 '30-34': '30-34',
 '35-39': '35-39',
 '40-44': '40-44',
 '45-49': '45-49',
 '5-9': '5-9',
 '50-54': '50-54'}

In [9]:
tumor_size_dict['0-4'] = '00-04'
tumor_size_dict['5-9'] = '05-09'

In [10]:
df['tumor-size'] = df['tumor-size'].map(tumor_size_dict)

Clean the column `inv-nodes`

In [11]:
inv_nodes_dict = dict(zip(list(df['inv-nodes'].unique()),list(df['inv-nodes'].unique())))

In [12]:
inv_nodes_dict['0-2']  = '00-02'
inv_nodes_dict['3-5']  = '03-05'
inv_nodes_dict['6-8']  = '06-08'
inv_nodes_dict['9-11'] = '09-11'

In [13]:
df['inv-nodes'] = df['inv-nodes'].map(inv_nodes_dict)

Inspect the data

In [14]:
df.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,00-02,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,00-02,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,00-02,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,00-02,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,00-04,00-02,no,2,right,right_low,no


Prepare the lists of numeric features, ordinal features, categorical features

In [44]:
categorical_features= df.dtypes.loc[df.dtypes == 'object'].index.values
print("the non-numeric featueres are:")
print(categorical_features)

the non-numeric featueres are:
['Class' 'age' 'menopause' 'tumor-size' 'inv-nodes' 'node-caps' 'breast'
 'breast-quad' 'irradiat']


In [45]:
numeric_features= list(set(df.dtypes.index.values)-set(categorical_features))
print("the numeric features are:")
print(numeric_features)

the numeric features are:
['deg-malig']


In [46]:
ordinal_features =['age', 'tumor-size','inv-nodes']
print("The ordinal features are:")
print(ordinal_features)

The ordinal features are:
['age', 'tumor-size', 'inv-nodes']


In [47]:
categorical_features= list(set(categorical_features) - set(ordinal_features)-set(['Class']))
print("the categorical features are:")
print(categorical_features)

the categorical features are:
['breast', 'irradiat', 'breast-quad', 'node-caps', 'menopause']


Prepare the transformer

In [48]:
 # transf_dtype = np.float64
transf_dtype = np.int32

categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse = False, dtype = transf_dtype)
ordinal_transformer = OrdinalEncoder(dtype = transf_dtype)
preprocessor = ColumnTransformer(
    transformers = [('cat', categorical_transformer, categorical_features),
                    ('ord', ordinal_transformer, ordinal_features)
                   ],
                    remainder = 'passthrough'
    )

Split X and y and check the shapes

In [22]:
X = df.drop(['Class'], axis = 1)
y = df['Class']

In [49]:
labels = y.unique()
print("The labels are")
print(labels)

The labels are
['no-recurrence-events' 'recurrence-events']


In [50]:
X.shape

(286, 9)

Fit the preprocessor with X and check the parameters printing the `.named_transformers_` attribute

In [51]:
preprocessor.fit(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 OneHotEncoder(dtype=<class 'numpy.int32'>,
                                               handle_unknown='ignore',
                                               sparse=False),
                                 ['breast', 'irradiat', 'breast-quad',
                                  'node-caps', 'menopause']),
                                ('ord',
                                 OrdinalEncoder(dtype=<class 'numpy.int32'>),
                                 ['age', 'tumor-size', 'inv-nodes'])])

In [52]:
print(preprocessor.named_transformers_)

{'cat': OneHotEncoder(dtype=<class 'numpy.int32'>, handle_unknown='ignore',
              sparse=False), 'ord': OrdinalEncoder(dtype=<class 'numpy.int32'>), 'remainder': 'passthrough'}


Fit-transform X and store the result in X_p, check the shape

In [54]:
X_p = preprocessor.fit_transform(X)

In [55]:
X_p.shape

(286, 20)

For ease of inspection transform `X_p` into a data frame `df_p` and inspect it

In [56]:
df_p =pd.DataFrame(X_p)

In [57]:
df_p.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000,286.000000
mean,0.531469,0.468531,0.762238,0.237762,0.003497,0.073427,0.384615,0.339161,0.083916,0.115385,0.027972,0.776224,0.195804,0.451049,0.024476,0.524476,2.664336,4.881119,0.517483,2.048951
std,0.499883,0.499883,0.426459,0.426459,0.059131,0.261293,0.487357,0.474254,0.277748,0.320046,0.165182,0.417504,0.397514,0.498470,0.154791,0.500276,1.011818,2.105930,1.110417,0.738217
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,4.000000,0.000000,2.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,5.000000,0.000000,2.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,3.000000,6.000000,1.000000,3.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,10.000000,6.000000,3.000000


In [58]:
df_p.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,6,0,3
1,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,2,4,0,2
2,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,2,4,0,2
3,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,4,3,0,2
4,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,1,2,0,0,2


The columns in the transformed dataset are generated according to the order you see printing the preprocessor after fitting, therefore the last four columns correspond to `'age', 'tumor-size', 'inv-nodes', 'deg-malig'`.

In order to inspect if the translation and check if the mapping is as expected, compare the sorted values of df['tumor-size'] and df_p[17], e.g. comparing the index sequences

In [60]:
orig_col= 'tumor-size'
transf_col= 17
a= pd.DataFrame(zip(df[orig_col],df_p[transf_col]))
print('The number of index discordances between \'{}\' and \'{}\' is {}'.\
      format(orig_col, transf_col, sum(a.sort_values(by = 0).index.values!=a.sort_values(by = 1).index.values)))

The number of index discordances between 'tumor-size' and '17' is 0


Train/test split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_p,y, random_state = random_state)

Classification and test

In [62]:
model_lbls = [
             'dt', 
             'nb', 
             'lp', 
             'svc', 
              'knn',
             'rfc',
             'ada',
            ]

# Set the parameters by cross-validation
tuned_param_dt = [{'max_depth': list(range(1,20))}]
tuned_param_nb = [{'var_smoothing': [10**i for i in range(1,-11, -1)]}]
tuned_param_lp = [{'early_stopping': [True]}]
tuned_param_svc = [{'kernel': ['rbf'], 
                    'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000],
                    'C': [10**i for i in range(0,4)],
                    },
                    {'kernel': ['linear'],
                    'C': [10**i for i in range(0,4)],
                    },
                   ]
tuned_param_knn =[{'n_neighbors': list(range(1,11)),
                   'metric': ['euclidean', 'manhattan', 'chebyshev']}
                 ]
tuned_param_rfc =[{'max_depth': list(range(1,11))}]
tuned_param_ada = [{'learning_rate': [1., 0.1, 0.01, 0.001, 0.0001]}]

models = {
    'dt': {'name': 'Decision Tree       ',
           'estimator': DecisionTreeClassifier(), 
           'param': tuned_param_dt,
          },
    'nb': {'name': 'Gaussian Naive Bayes',
           'estimator': GaussianNB(),
           'param': tuned_param_nb
          },
    'lp': {'name': 'Linear Perceptron   ',
           'estimator': Perceptron(),
           'param': tuned_param_lp,
          },
    'svc':{'name': 'Support Vector      ',
           'estimator': SVC(), 
           'param': tuned_param_svc
          },
    'knn':{'name': 'K Nearest Neighbor  ',
           'estimator': KNeighborsClassifier(),
           'param': tuned_param_knn
          },
    'rfc':{'name': 'Random Forest       ',
           'estimator': RandomForestClassifier(),
           'param': tuned_param_rfc
          }, 
    'ada':{'name': 'Adaboost            ',
           'estimator': AdaBoostClassifier(),
           'param': tuned_param_ada
          },
}

scores = [
#    'precision_macro', 
    'recall_macro', 
#    'accuracy',
#    'f1_macro'
]

In [63]:
# def plot_confusion_matrix(cm):
#     print(cm) 
#     fig = plt.figure(figsize=(10,10))
#     ax = fig.add_subplot(111) 
#     cax = ax.matshow(cm) 
#     plt.title('Confusion matrix of the classifier') 
#     fig.colorbar(cax) 
#     ax.set_xticklabels([''] + labels) 
#     ax.set_yticklabels([''] + labels) 
#     plt.xlabel('Predicted') 
#     plt.ylabel('True') 
#     plt.show()

def print_results(model):
    print("Best parameters set found on train set:")
    print()
    # if best is linear there is no gamma parameter
    print(model.best_params_)
    print()
    print("Grid scores on train set:")
    print()
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    params = model.cv_results_['params']
    for mean, std, params_tuple in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params_tuple))
    print()
    print("Detailed classification report for the best parameter set:")
    print()
    print("The model is trained on the full train set.")
    print("The scores are computed on the full test set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)
    print(classification_report(y_true, y_pred))
    cm = confusion_matrix(y_true,y_pred, labels = labels)
    print(cm)
#     plot_confusion_matrix(cm)
    print()

In [65]:
results_short = {}

for score in scores:
    print('='*40)
    print("# Tuning hyper-parameters for %s" % score)
    print()

    #'%s_macro' % score ## is a string formatting expression
    # the parameter after % is substituted in the string placeholder %s
    for m in model_lbls:
        print('-'*40)
        print("Trying model {}".format(models[m]['name']))
        clf = GridSearchCV(models[m]['estimator'], models[m]['param'], cv=5,
                           scoring=score, 
                           #iid = False, 
                           return_train_score = False,
                           n_jobs = 2, # this allows using multi-cores
                           )
        clf.fit(X_train, y_train)
        print_results(clf)
        results_short[m] = clf.best_score_
    print("Summary of results for {}".format(score))
    print("Estimator")
    for m in results_short.keys():
        print("{}\t - score: {:4.2}%".format(models[m]['name'], results_short[m]))

# Tuning hyper-parameters for recall_macro

----------------------------------------
Trying model Decision Tree       
Best parameters set found on train set:

{'max_depth': 2}

Grid scores on train set:

0.567 (+/-0.073) for {'max_depth': 1}
0.601 (+/-0.119) for {'max_depth': 2}
0.590 (+/-0.127) for {'max_depth': 3}
0.570 (+/-0.114) for {'max_depth': 4}
0.535 (+/-0.113) for {'max_depth': 5}
0.571 (+/-0.134) for {'max_depth': 6}
0.577 (+/-0.113) for {'max_depth': 7}
0.560 (+/-0.261) for {'max_depth': 8}
0.593 (+/-0.259) for {'max_depth': 9}
0.567 (+/-0.129) for {'max_depth': 10}
0.559 (+/-0.221) for {'max_depth': 11}
0.551 (+/-0.153) for {'max_depth': 12}
0.576 (+/-0.171) for {'max_depth': 13}
0.592 (+/-0.156) for {'max_depth': 14}
0.588 (+/-0.198) for {'max_depth': 15}
0.544 (+/-0.213) for {'max_depth': 16}
0.524 (+/-0.191) for {'max_depth': 17}
0.556 (+/-0.251) for {'max_depth': 18}
0.556 (+/-0.254) for {'max_depth': 19}

Detailed classification report for the best parameter set:

Th